In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
registry_df = pd.read_csv('DataSets/registry_game.csv')
product_df = pd.read_csv('DataSets/product_loaded.csv')
ean_df = pd.read_csv('DataSets/product_ean_conversion.csv')
missions_df = pd.read_csv('DataSets/missions_players.csv')
accesses_df = pd.read_csv('DataSets/app_accesses.csv')
awards_df = pd.read_csv('DataSets/awards_moms.csv')


In [3]:
"""
print(registry_df.head())
print(product_df.head())
print(ean_df.head())
print(missions_df.head())
print(accesses_df.head())
print(awards_df.head())
"""

'\nprint(registry_df.head())\nprint(product_df.head())\nprint(ean_df.head())\nprint(missions_df.head())\nprint(accesses_df.head())\nprint(awards_df.head())\n'

In [ ]:
print(registry_df.describe())
print(product_df.describe())
print(ean_df.describe())
print(missions_df.describe())
print(accesses_df.describe())
print(awards_df.describe())

In [4]:
print(registry_df.info())
print('\n')
print(product_df.info())
print('\n')
print(ean_df.info())
print('\n')
print(missions_df.info())
print('\n')
print(accesses_df.info())
print('\n')
print(awards_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45971 entries, 0 to 45970
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id_player            45971 non-null  int64  
 1   DtaRegUserData       45971 non-null  object 
 2   DtaPresuntoParto     43105 non-null  object 
 3   ETA_MM_BambinoTODAY  43105 non-null  float64
 4   ETA_MM_BambinoREG    43103 non-null  float64
 5   Provincia            45697 non-null  object 
 6   SiglaProvincia       43069 non-null  object 
 7   Comune               24642 non-null  object 
 8   Regione              45677 non-null  object 
dtypes: float64(2), int64(1), object(6)
memory usage: 3.2+ MB
None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596529 entries, 0 to 1596528
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   id_player      1596529 non-null  int64  
 1   missionDetail  501

In [ ]:
## percentage of missing values in each dataframe

print(registry_df.isnull().sum() / registry_df.shape[0] * 100)
print('\n')
print(product_df.isnull().sum() / product_df.shape[0] * 100)
print('\n')
print(ean_df.isnull().sum() / ean_df.shape[0] * 100)
print('\n')
print(missions_df.isnull().sum() / missions_df.shape[0] * 100)
print('\n')
print(accesses_df.isnull().sum() / accesses_df.shape[0] * 100)
print('\n')
print(awards_df.isnull().sum() / awards_df.shape[0] * 100)

In [5]:
registry_df = registry_df.dropna()
registry_df = registry_df[registry_df['ETA_MM_BambinoREG'] >= 0]

In [6]:
product_df['missionDetail'] = product_df['missionDetail'].fillna('No mission')
product_df['EAN'] = pd.to_numeric(product_df['EAN'], errors='coerce')

In [7]:
ean_df['TIER'] = ean_df['TIER'].fillna('Unknown')

In [8]:
missions_df['points'] = missions_df['points'].fillna(0)

In [9]:
accesses_df['source'] = accesses_df['source'].fillna('Unknown')

In [10]:
# Drop rows with missing values
awards_df = awards_df.dropna()
print(awards_df.head())

# # Convert date columns to datetime objects
# awards_df['datarichiestapremio'] = pd.to_datetime(awards_df['datarichiestapremio'], format='%Y-%m-%d')
# awards_df['updated_at'] = pd.to_datetime(awards_df['updated_at'], format='%Y-%m-%d %H:%M:%S')

  datarichiestapremio  id_player                 nomepremio  puntipremio  \
0          2022-01-20     1672.0  Bicicletta Personalizzata        21000   
1          2021-10-20   537088.0  Velocipede Personalizzato        18000   
2          2021-10-08   537088.0  Velocipede Personalizzato        18000   
3          2022-01-27  1461448.0       Buono Sconto Photosì         3000   
4          2021-12-28  1633286.0     Buono Sconto Leolandia            0   

  tipopremio formatPremio deliveryMode  
0      basic     physical         home  
1      basic     physical         home  
2      basic     physical         home  
3       gift      digital        email  
4    special      digital        email  


In [11]:
# Merge product_df with conversion_df
product_df = product_df.merge(ean_df, on='EAN')

# Merge missions_df with product_df
missions_df = missions_df.merge(product_df, on='id_player')

# Calculate the total points earned by each player
total_points_df = missions_df.groupby('id_player')['points_x'].sum().reset_index()
total_points_df.columns = ['id_player', 'total_points']

# Merge registry_df with total_points_df
registry_df = registry_df.merge(total_points_df, on='id_player')

# Calculate the number of app accesses by each player
accesses_count_df = accesses_df.groupby('id_player')['updated_at'].count().reset_index()
accesses_count_df.columns = ['id_player', 'accesses_count']

# Merge registry_df with accesses_count_df
registry_df = registry_df.merge(accesses_count_df, on='id_player')

# Calculate the number of awards requested by each player
awards_count_df = awards_df.groupby('id_player')['datarichiestapremio'].count().reset_index()
awards_count_df.columns = ['id_player', 'awards_count']

# Merge registry_df with awards_count_df
registry_df = registry_df.merge(awards_count_df, on='id_player')

# Display the cleaned and transformed registry_df
print(registry_df.head())

   id_player DtaRegUserData DtaPresuntoParto  ETA_MM_BambinoTODAY  \
0      72728     2020-01-25       2019-12-18                 33.0   
1     948474     2021-02-23       2020-06-03                 27.0   
2     489194     2020-09-25       2020-04-28                 29.0   
3     448656     2020-08-19       2020-08-01                 25.0   
4      34600     2019-12-05       2019-11-01                 34.0   

   ETA_MM_BambinoREG Provincia SiglaProvincia             Comune    Regione  \
0                1.0   BELLUNO             BL            BELLUNO     VENETO   
1                8.0      ASTI             AT               ASTI   PIEMONTE   
2                4.0    MILANO             MI            VANZAGO  LOMBARDIA   
3                0.0   VICENZA             VI  CORNEDO VICENTINO     VENETO   
4                1.0  BRINDISI             BR           BRINDISI     PUGLIA   

   total_points  accesses_count  awards_count  
0       49200.0              14             1  
1       61600.

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
accesses_df['source'] = le.fit_transform(accesses_df['source'])

# Convert 'updated_at' column in accesses_df to datetime format
accesses_df['updated_at'] = pd.to_datetime(accesses_df['updated_at'])

# Visualize outliers in accesses_df dataset
plt.figure(figsize=(8, 6))
sns.boxplot(x=accesses_df['source'], y=accesses_df['updated_at'])
plt.xlabel('Source')
plt.ylabel('Date and Time of Connection')
plt.title('Outliers in App Accesses')
plt.show()


KeyboardInterrupt: 

In [ ]:
# Visualize outliers in missions_df dataset
plt.figure(figsize=(8, 6))
sns.boxplot(x=missions_df['points'])
plt.xlabel('Points Gained in the Mission')
plt.title('Outliers in Missions Players')
plt.show()


In [ ]:
# Visualize outliers in product_df dataset
plt.figure(figsize=(8, 6))
sns.boxplot(x=product_df['points'])
plt.xlabel('Points Gained for that Product')
plt.title('Outliers in Product Loaded')
plt.show()


In [ ]:
# Visualize outliers in awards_df dataset
plt.figure(figsize=(8, 6))
sns.boxplot(x=awards_df['puntipremio'])
plt.xlabel('Points Required to Obtain the Prize')
plt.title('Outliers in Awards Moms')
plt.show()